In [13]:
import requests
import os
import re
import json
import time
from datetime import datetime
from bs4 import BeautifulSoup


In [4]:
""""
准备干一波凤凰网的新闻, 要构建一个层级的新闻类别, 目前暂定为两级, 每个小级别应该要有 1000 条数据, 不知道能不能达成, 我看有些层级基本没数据
"""
label_dict = {"政务": ["高层动态", "政策", "人事", "地方", "发展治理", "反腐", "环保", "文旅"]}

In [7]:
import requests

cookies = {
    "userid": "1647525251481_gkn7z47154",
    "if_prov": "cn0571",
    "if_city": "0571",
    "prov": "cn0571",
    "city": "0571",
    "weather_city": "zj_hz",
    "region_ip": "125.120.227.x",
    "region_ver": "1.2",
}

headers = {
    "Connection": "keep-alive",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    "sec-ch-ua-mobile": "?0",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "sec-ch-ua-platform": '"Windows"',
    "Accept": "*/*",
    "Sec-Fetch-Site": "same-site",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Dest": "script",
    "Referer": "https://gov.ifeng.com/shanklist/22-35143-",
    "Accept-Language": "zh-CN,zh;q=0.9",
}

params = (
    ("callback", "getColumnInfoCallback"),
    ("_", "16475255953401"),
)

response = requests.get(
    "https://shankapi.ifeng.com/shanklist/_/getColumnInfo/_/default/6879595161253318693/1640225010000/100/22-35143-/getColumnInfoCallback",
    headers=headers,
    params=params,
    cookies=cookies,
)

# print(response.text)
json_data = json.loads(response.text.replace("getColumnInfoCallback(", "")[:-1])
print(len(json_data["data"]["newsstream"]))


100


In [ ]:
"""
url 的构建方式
"""
last_news_id = "6811416760466543017"
last_time = int(time.mktime(time.strptime("2021-06-18 07:03:10", "%Y-%m-%d %H:%M:%S")) * 1000)
now_time = int(time.time() * 10000)
url = "https://shankapi.ifeng.com/shanklist/_/getColumnInfo/_/default/{last_news_id}/{last_time}/20/22-35146-/getColumnInfoCallback?callback=getColumnInfoCallback&_={now_time}"
url = url.format(last_news_id=last_news_id, last_time=last_time, now_time=now_time)
url


In [ ]:
response = requests.get(url, headers=headers, cookies=cookies)
response.text


In [11]:
"""
base62 转换方式
"""
BASE62 = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

def encode_b62(num, alphabet=BASE62):
    """Encode a positive number in Base X
    Arguments:
    - `num`: The number to encode
    - `alphabet`: The alphabet to use for encoding
    """
    if num == 0:
        return alphabet[0]
    arr = []
    base = len(alphabet)
    while num:
        num, rem = divmod(num, base)
        arr.append(alphabet[rem])
    arr.reverse()
    return ''.join(arr)

def decode_b62(string, alphabet=BASE62):
    """Decode a Base X encoded string into the number
    Arguments:
    - `string`: The encoded string
    - `alphabet`: The alphabet to use for encoding
    """
    base = len(alphabet)
    strlen = len(string)
    num = 0
    idx = 0
    for char in string:
        power = (strlen - (idx + 1))
        num += alphabet.index(char) * (base ** power)
        idx += 1

    return num

encode_b62(6764717602024464456) == "83iTeuiRl4i"

True

In [60]:
def parse_one_api(url, result):
    response = requests.get(url, headers=headers, cookies=cookies)
    text = response.text.replace("getColumnInfoCallback(", "")[:-1]
    json_data = json.loads(text)
    code = json_data["code"]
    if code != 0:
        print(json_data)
        return False, None, None
    data = json_data["data"]
    is_end = data["isEnd"]
    for item in data["newsstream"]:
        result.append(item["title"])
    
    last_news_id = data["newsstream"][-1]["id"]
    last_time = data["newsstream"][-1]["newsTime"]
    return is_end, last_news_id, last_time


def build_new_url(last_news_id, last_time, category_id="22-35146-"):
    last_time = int(time.mktime(time.strptime(last_time, "%Y-%m-%d %H:%M:%S")) * 1000)
    now_time = int(time.time() * 10000)
    url = "https://shankapi.ifeng.com/shanklist/_/getColumnInfo/_/default/{last_news_id}/{last_time}/100/{category_id}/getColumnInfoCallback?callback=getColumnInfoCallback&_={now_time}"
    url = "https://shankapi.ifeng.com/shanklist/_/getColumnInfo/_/selectedPool/{last_news_id}/{last_time}/100/{category_id}/getColumnInfoCallback?callback=getColumnInfoCallback&_={now_time}"
    url = url.format(last_news_id=last_news_id, last_time=last_time, now_time=now_time, category_id=category_id)
    return url

In [14]:
url = build_new_url(decode_b62("8EGQmMbzd1q"), "2022-03-10 10:09:15", "22-35143-")
result = []
parse_one_api(url, result)
print(len(result))

100


In [24]:
result = []
url = build_new_url(decode_b62("8EGQmMbzd1q"), "2022-03-10 10:09:15", "22-35143-")
is_end, last_news_id, last_time = parse_one_api(url, result)
i = 10
while i > 0:
    i -= 1
    if is_end:
        print("没有更多了")
        break
    url = build_new_url(last_news_id, last_time, "22-35143-")
    is_end, last_news_id, last_time = parse_one_api(url, result)
    print(is_end, last_news_id, last_time)


True 6735212528927768667 2020-11-20 00:12:40
没有更多了


In [29]:
def run_one_category(start_id, start_time, category_id, label):
    start_time = start_time.replace("年", "-").replace("月", "-").replace("日", "")
    result = []
    url = build_new_url(decode_b62(start_id), start_time, category_id)
    is_end, last_news_id, last_time = parse_one_api(url, result)
    i = 10
    while i > 0:
        i -= 1
        if is_end:
            print("没有更多了")
            break
        url = build_new_url(last_news_id, last_time, category_id)
        is_end, last_news_id, last_time = parse_one_api(url, result)
        print(is_end, last_news_id, last_time)
    print(len(result))
    with open(label + ".txt", "w", encoding="utf-8") as f:
        for item in result:
            f.write(f"{label}\t{item}\n")


In [62]:
run_one_category("8ETkeg6ilPY", "2022年03月19日 03:10:58", "20003", "科技_手机")

False 6902522869654237729 2022-02-24 23:22:57
True 6899875555567472697 2022-02-17 08:45:14
没有更多了
296
